# 

# <center>Web Scraping

In [1]:
# Imports
import re
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [2]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "FS" --iversions

Author: FS

pandas  : 1.2.2
re      : 2.2.1
json    : 2.0.9
requests: 2.27.1



In [3]:
# url
url = 'https://www.ivalor.com.br/empresas/listagem?p={}'
pages = list(range(1,30,1))

In [4]:
def scrapSectorsTable():
    
    lista_linhas = []
    
    for page in pages:

        # Return HTML
        resposta = requests.get(url.format(page), headers={'User-Agent': 'Custom'})
        bruto = BeautifulSoup(resposta.text, 'html.parser')

        # Filters
        dados_brutos = bruto.find('tbody', id = 'tab_empresas_body')
        linhas = dados_brutos.find_all('tr')
        
        for linha in linhas:
            cells = linha.find_all('td')
            str_cells = str(cells)
            clean1 = re.compile('<.*?>')
            clean = re.sub(clean1, '', str_cells)
            clean = re.sub(r'\xa0', '', clean)
            clean = re.sub(r"^\s+" , "" , clean)
            clean = re.sub(r'\[', '', clean)
            clean = re.sub(r'\]', '', clean)
            
            lista_linhas.append(clean)
            
    # Create dataframe
    df = pd.DataFrame(lista_linhas)
        
    # Expand string
    df1 = df[0].str.split(',', expand = True)
        
    # Extract labels
    col_labels = bruto.find_all('th')
    all_header = []
    col_str = str(col_labels)
    cleantext2 = BeautifulSoup(col_str, "lxml").get_text()
    all_header.append(cleantext2)
    df_head = pd.DataFrame(all_header)
    df_head1 = df_head[0].str.split(',', expand = True)
    
    # More cleaning
    df_head1[0] = df_head1[0].str.strip('[')
    df_head1[9] = df_head1[9].str.strip(']')
   
    # Concat dataframe 
    df_full = pd.concat([df_head1, df1]).reset_index(drop = True)
    df_full = df_full.rename(columns = df_full.iloc[0]).drop(df_full.index[[0]])
    
    # More cleaning
    df_full.columns = df_full.columns.str.strip(' ')
    for col in df_full.columns.to_list():
        df_full[col] = [x.strip(' ') for x in df_full[col]]
        
    return df_full

In [5]:
# Set df
df_sectors = scrapSectorsTable().iloc[:,2:]

In [6]:
# View
df_sectors

,Empresa,Ações,Classificação,Listagem,Tag Along,Setor,Subsetor,Segmento
1,ALLIAR,AALR3,Small Cap,Novo Mercado,100% para ações ON,Saúde,Serv.Méd.Hospit..Análises e Diagnósticos,Serv.Méd.Hospit..Análises e Diagnósticos
2,ABC BRASIL,ABCB4,Small Cap,Nível 2 de Governança Corporativa,100% para ações ON e PN,Financeiro,Intermediários Financeiros,Bancos
3,AMBEV S/A,ABEV3,Mega Cap,Balcão Org. Tradicional,80% para ações ON,Consumo não Cíclico,Bebidas,Cervejas e Refrigerantes
4,ADVANCED-DH,ADHM3,Nano Cap,Balcão Org. Tradicional,80% para ações ON,Saúde,Serv.Méd.Hospit..Análises e Diagnósticos,Serv.Méd.Hospit..Análises e Diagnósticos
5,AERIS,AERI3,Small Cap,Novo Mercado,100% para ações ON,Bens Industriais,Máquinas e Equipamentos,Máq. e Equip. Industriais
...,...,...,...,...,...,...,...,...
422,WHIRLPOOL,WHRL3 WHRL4,Mid Cap,Balcão Org. Tradicional,80% para ações ON,Consumo Cíclico,Utilidades Domésticas,Eletrodomésticos
423,WIZ S.A.,WIZS3,Small Cap,Novo Mercado,100% para ações ON,Financeiro,Previdência e Seguros,Corretoras de Seguros e Resseguros
424,WLM IND COM,WLMM3 WLMM4,Micro Cap,Balcão Org. Tradicional,80% para ações ON,Bens Industriais,Comércio,Material de Transporte
425,GX TLMEDC DH,WSON33,-,BDR Nível 3,None,Não Classificados,Não Classificado,Não Classificados


___